<a href="https://colab.research.google.com/github/harnalashok/deeplearning-sequences/blob/main/stock_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
# 0.0 Install attention
! pip install attention

In [113]:
# 1.0 Usual libraries
import numpy as np
import pandas as pd

# 1.1 tensorflow related libraries
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import load_model, Model
from attention import Attention

In [30]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [45]:
path = "/content/drive/MyDrive/Colab_data_files/tatamotors/tatamotors.csv"
f = open(path)

In [46]:
lines = f.read()

In [47]:
lines[:4]

'date'

In [48]:
lines = lines.split("\n")

In [49]:
lines[:4]

['date,symbol,series,prevclose,open,high,low,last,close,vwap,volume,turnover',
 '03-01-2000,TELCO,EQ,201.6,207.4,217.25,207.4,217,216.75,214.28,676126,1.44878E+13',
 '04-01-2000,TELCO,EQ,216.75,217,219,206,211.9,208.2,209.5,679215,1.42296E+13',
 '05-01-2000,TELCO,EQ,208.2,194,217.8,194,213.1,213.25,210.33,1120951,2.35768E+13']

In [50]:
header = lines[0]
header = header.split(',')
print(header)

['date', 'symbol', 'series', 'prevclose', 'open', 'high', 'low', 'last', 'close', 'vwap', 'volume', 'turnover']


In [51]:
lines = lines[1:]

In [52]:
lines[:4]
len(lines)   # 5307

['03-01-2000,TELCO,EQ,201.6,207.4,217.25,207.4,217,216.75,214.28,676126,1.44878E+13',
 '04-01-2000,TELCO,EQ,216.75,217,219,206,211.9,208.2,209.5,679215,1.42296E+13',
 '05-01-2000,TELCO,EQ,208.2,194,217.8,194,213.1,213.25,210.33,1120951,2.35768E+13',
 '06-01-2000,TELCO,EQ,213.25,215,229.9,215,222,222.1,225.29,1968998,4.43593E+13']

5307

In [ ]:
np.zeros([12,12])

In [67]:
data = np.zeros([len(lines), len(header)-3])
data.shape

(5307, 9)

In [81]:
 for i, j in enumerate(lines):
   j = j.split(',')
   #print(j[3:])
   #break;
   t = np.array(j[3:])
   data[i] = t
   break



In [84]:
for i in range(len(lines)-1):
  j = lines[i].split(',')
  t = np.array(j[3:])
  data[i] = t

In [85]:
data.shape

(5307, 9)

In [164]:
data = pd.read_csv(path)
data = data.loc[:,['prevclose', 'open', 'high', 'low', 'last', 'vwap', 'volume', 'turnover', 'close']]
data = data.values

In [165]:
data.shape

(5306, 9)

In [221]:
# 3.1 Global mean of each one of all eight columns
mean = data.mean(axis = 0)
data -= mean


In [222]:
# Std of each one of eight columns
std = data.std(axis = 0)
data /= std

In [120]:
np.zeros(3)

array([0., 0., 0.])

In [206]:
def train_gen(time_steps, batches, min_index,max_index, ahead, data):
  while 1:
    dx = np.zeros([batches,time_steps,data.shape[-1]])
    dy = np.zeros(batches)
    for i in range(batches):
      start = np.random.randint(min_index,max_index)
      #print(start)
      df = data[start:start+time_steps, :]
      #print(df.shape)
      dx[i,:,:] = df
      dy[i] = data[start+time_steps+1,8 ]
    yield dx, dy
  


In [290]:
time_steps = 70
min_index = 0
max_index = 4000
ahead = 1
batches = 32
data = data

In [291]:
t = train_gen(time_steps, batches, min_index,max_index, ahead, data)

In [292]:
X,y = next(t)

In [293]:
X.shape
y.shape

(32, 70, 9)

(32,)

In [294]:
time_steps = 70
min_index = 4000
max_index = 5000
ahead = 1
batches = 32
data = data

In [295]:
v = train_gen(time_steps, batches, min_index,max_index, ahead, data)

In [296]:
# 4.0 Define the model.
input_dim = X.shape[-1]
model_input = Input(shape=(time_steps, input_dim))
x = LSTM(70, return_sequences=True)(model_input)
x = LSTM(70, return_sequences= True)(x)
x = Attention(32)(x)
x = Dense(16)(x)
x = Dense(1)(x)
model = Model(model_input, x)

In [297]:
# 4.1 Compile and print model summary
model.compile(loss='mean_squared_error', optimizer='adam')
print(model.summary())

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 70, 9)]      0                                            
__________________________________________________________________________________________________
lstm_13 (LSTM)                  (None, 70, 70)       22400       input_10[0][0]                   
__________________________________________________________________________________________________
lstm_14 (LSTM)                  (None, 70, 70)       39480       lstm_13[0][0]                    
__________________________________________________________________________________________________
last_hidden_state (Lambda)      (None, 70)           0           lstm_14[0][0]                    
____________________________________________________________________________________________

In [298]:
H = model.fit(
              t,
              steps_per_epoch= 60,
              validation_data = v,
              validation_steps = 10,
              epochs= 100)

Epoch 1/100
60/60 [==============================] - 9s 100ms/step - loss: 0.0946 - val_loss: 0.0305
Epoch 2/100
60/60 [==============================] - 5s 86ms/step - loss: 0.0163 - val_loss: 0.0142
Epoch 3/100
60/60 [==============================] - 5s 86ms/step - loss: 0.0176 - val_loss: 0.0351
Epoch 4/100
60/60 [==============================] - 5s 86ms/step - loss: 0.0146 - val_loss: 0.0129
Epoch 5/100
60/60 [==============================] - 5s 86ms/step - loss: 0.0196 - val_loss: 0.0141
Epoch 6/100
60/60 [==============================] - 5s 87ms/step - loss: 0.0172 - val_loss: 0.0093
Epoch 7/100
60/60 [==============================] - 5s 87ms/step - loss: 0.0099 - val_loss: 0.0081
Epoch 8/100
60/60 [==============================] - 5s 87ms/step - loss: 0.0083 - val_loss: 0.0104
Epoch 9/100
60/60 [==============================] - 5s 88ms/step - loss: 0.0099 - val_loss: 0.0062
Epoch 10/100
60/60 [==============================] - 5s 87ms/step - loss: 0.0106 - val_loss: 0.006

In [273]:
xt = train_gen(time_steps, batches, min_index,max_index, ahead, data)

In [275]:
r,u= next(xt)

In [276]:
r.shape
u.shape

(32, 70, 9)

(32,)

In [280]:
result = model.evaluate(
                         xt, 
                         steps = 140   # Total number of steps (batches of samples) before declaring the evaluation round finished.
                        )

140/140 [==============================] - 2s 12ms/step - loss: 0.0030


In [279]:
result

0.0032031990122050047

In [282]:
input_dim = X.shape[-1]
model_input1 = Input(shape=(time_steps, input_dim))
x = LSTM(70, return_sequences=True)(model_input1)
x = LSTM(70, return_sequences=False)(x)
x = Dense(35)(x)
x = Dense(1)(x)
model1 = Model(model_input1, x)

In [283]:
# 4.1 Compile and print model summary
model1.compile(loss='mean_squared_error', optimizer='adam')
print(model1.summary())

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 70, 9)]           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 70, 70)            22400     
_________________________________________________________________
lstm_8 (LSTM)                (None, 70)                39480     
_________________________________________________________________
dense_7 (Dense)              (None, 35)                2485      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 36        
Total params: 64,401
Trainable params: 64,401
Non-trainable params: 0
_________________________________________________________________
None


In [284]:
H1 = model1.fit(
              t,
              steps_per_epoch= 60,
              validation_data = v,
              validation_steps = 10,
              epochs= 100)

Epoch 1/100
60/60 [==============================] - 9s 98ms/step - loss: 0.0299 - val_loss: 0.0036
Epoch 2/100
60/60 [==============================] - 5s 81ms/step - loss: 0.0041 - val_loss: 0.0035
Epoch 3/100
60/60 [==============================] - 5s 83ms/step - loss: 0.0036 - val_loss: 0.0028
Epoch 4/100
60/60 [==============================] - 5s 82ms/step - loss: 0.0035 - val_loss: 0.0031
Epoch 5/100
60/60 [==============================] - 5s 82ms/step - loss: 0.0031 - val_loss: 0.0034
Epoch 6/100
60/60 [==============================] - 5s 83ms/step - loss: 0.0029 - val_loss: 0.0032
Epoch 7/100
60/60 [==============================] - 5s 83ms/step - loss: 0.0029 - val_loss: 0.0032
Epoch 8/100
60/60 [==============================] - 5s 83ms/step - loss: 0.0029 - val_loss: 0.0033
Epoch 9/100
60/60 [==============================] - 5s 83ms/step - loss: 0.0024 - val_loss: 0.0030
Epoch 10/100
60/60 [==============================] - 5s 82ms/step - loss: 0.0025 - val_loss: 0.0026